In [ ]:
import cv2
import numpy as np
from yolov8 import YOLOv8
from byte_track import ByteTrack

In [ ]:
model_file = "/home/cds-jetson-host/ultralytics/yolov8n-seg-1class.yaml"
weights_file = "/home/cds-jetson-host/ultralytics/runs/segment/train2/weights/last.pt"
yolov8 = YOLOv8(model_file, weights_file, min_score=0.7)
byte_track = ByteTrack(frame_rate=30, track_buffer=30)

In [ ]:
video_file = "dataset/all/VID_20230626_183354.mp4"
video = cv2.VideoCapture(video_file)

cv2.namedWindow("image", cv2.WINDOW_NORMAL)
cv2.namedWindow("segmentation", cv2.WINDOW_NORMAL)
cv2.namedWindow("tracking", cv2.WINDOW_NORMAL)
cv2.resizeWindow("image", 600, 600)
cv2.resizeWindow("segmentation", 600, 600)
cv2.resizeWindow("tracking", 600, 600)
while True:
    ret, image = video.read()
    if ret:
        scores, classes_ids, boxes, masks = yolov8.segment(image)
        segmentation_vis = image.copy()
        YOLOv8.draw_detections(segmentation_vis, scores, classes_ids, boxes, masks)
        segmentation_image = YOLOv8.to_segmentation_image(classes_ids, masks)
        boxes, scores, classes_ids, masks = ByteTrack.from_segmentation_image(segmentation_image)
        tracked_objects = byte_track.track(boxes, scores, classes_ids, masks=masks)
        tracking_vis = image.copy()
        ByteTrack.draw_tracked_objects(tracking_vis, tracked_objects)
        tracking_image = ByteTrack.to_tracking_image(tracked_objects, segmentation_image.shape)

        cv2.imshow("image", image)
        cv2.imshow("segmentation", segmentation_vis)
        cv2.imshow("tracking", tracking_vis)
        key = cv2.waitKey(100)
        if key != -1:
            break
    else:
        break
cv2.destroyWindow("image")
cv2.destroyWindow("segmentation")
cv2.destroyWindow("tracking")

video.release()